# A Danish Identity Phrase Templates Test Set

The goal of this notebook is to create a synthetic dataset (IPTTS) that can be used to measure the unintended bias in toxicity classifiers. Samples are created from selected verbs, adjectives and identity terms. 

Inspired by Dixon et al. (2018): https://doi.org/10.1145/3278721.3278729 

### Prepare functions, data etc.

In [2]:
# import libraries
import lemmy
import os
import pandas as pd
import re
from typing import Dict, List

In [3]:
# utility functions
def read_content(filename:str) -> List[str]:
    """Opens file and returns its contents as a list split by newlines.

    Args:
        filename (str): name of file.

    Returns:
        List[str]: list of the lines in the file.
    """
    f = open(filename)
    contents = f.read().split("\n")
    f.close()
    return contents

def get_word_pos(wordtype:str, wordlist:List[str]) -> int:
    """Returns the position of the word to be inserted in list of words.

    Args:
        wordtype (str): either verb, identity, article or adjective.
        wordlist (List[str]): the sentence template as a list of words.

    Returns:
        int: the index of the word to be inserted.
    """
    for i, word in enumerate(wordlist):
        if wordtype.upper() == "VERB":
            if "VERB" in word:
                return i
        elif wordtype.upper() == "IDENTITY":
            if "IDENTITY" in word:
                return i
        elif wordtype.upper() == "ARTICLE":
            if "ARTICLE" in word:
                return i
        elif wordtype.upper() == "ADJECTIVE":
            if "ADJECTIVE" in word:
                return i
        else:
            return None

def determine_verb_list(verbtype:str, polarity:str) -> List[str]:
    """Determines which verb list to use based on the verbtype specified in the sample
    and returns its contents.

    Args:
        verbtype (str): the options are "VERB_PRS" (present tense), "VERB_IMP" (imperative), or "VERB_PASS" (passive).
        polarity (str): either "neg" (negative) or "pos" (positive).

    Returns:
        List[str]: a list of either negative or positive verbs of the given type. 
    """
    if polarity == "neg":
        if verbtype == "VERB_PRS":
            return read_content("verbs/neg_prs.txt") # present tense
        elif verbtype == "VERB_IMP":
            return read_content("verbs/neg_imp.txt") # imperative
        elif verbtype == "VERB_PASS":
            return read_content("verbs/neg_pass.txt") # passive
    elif polarity == "pos":
        if verbtype == "VERB_PRS":
            return read_content("verbs/pos_prs.txt") # present tense
        elif verbtype == "VERB_IMP":
            return read_content("verbs/pos_imp.txt") # imperative
        elif verbtype == "VERB_PASS":
            return read_content("verbs/pos_pass.txt") # passive
    return None
    
def determine_identity_lists(identitytype:str) -> Dict[str, List[str]]:
    """Determines which type of identity list to use based on the type specified in the sample
    and returns the contents for each identity group as a dictionary.

    Args:
        identitytype (str): either "IDENTITY_PL" (plural) or "IDENTITY_SG" (singular).

    Returns:
        Dict[str: List[str]]: a dictionary of three lists of either singular or plural 
        identity words (female, male, and non-conform gender identities). 
    """    
    identitytype = re.sub(r'[^A-z]','', identitytype) # remove punctuation from identity type
    if identitytype == "IDENTITY_PL":
        return {
            "F": read_content("identities/female_pl.txt"), 
            "M": read_content("identities/male_pl.txt"), 
            "Q": read_content("identities/queer_pl.txt")
            }
    elif identitytype == "IDENTITY_SG":
        return {
            "F": read_content("identities/female_sg.txt"), 
            "M": read_content("identities/male_sg.txt"), 
            "Q": read_content("identities/queer_sg.txt")
            }    
    else:
        return None

In [4]:
# main function
def generate_samples(templates:List[str], polarity:str=None) -> List[str]:
    """
    Generate samples from templates by slotting in identities and if necessary, verbs and adjectives. Returns a list of the generated samples.
    
    Args:
        templates (List[str]): list of templates as strings.
        polarity (str): allows the values "pos" and "neg" that specifies whether the sentiment of the verbs/adjectives is positive or negative. 

    Returns:
        List[str]: list of generated samples as strings.
    """
    generated_samples = []
    samples_groups = []
    samples_identity_terms = []

    # go through templates
    for template in templates:

        sample = template.split()
        
        # get identity position and list
        IDENTITY_POS = get_word_pos("identity", sample)
        IDENTITY_LIST_DICT = determine_identity_lists(sample[IDENTITY_POS])
        IS_PLURAL = sample[IDENTITY_POS].endswith("_PL") # check for plurality

        # get verb position and list (if relevant)
        VERB_POS = get_word_pos("verb", sample)
        if VERB_POS is not None: # i.e. if there's a verb slot in the template
            VERB_LIST = determine_verb_list(sample[VERB_POS], polarity)
        
        # get article position
        ART_POS = get_word_pos("article", sample)
        
        # get adjective position
        ADJ_POS = get_word_pos("adjective", sample)

        # create one version of this template per identity
        for identity_group in IDENTITY_LIST_DICT: # loops through keys (F = female, M = male, Q = queer/non-conform gender identities)
            for identity in IDENTITY_LIST_DICT[identity_group]: # loops through identity terms in list for this group, e.g. female
                identity_copy = identity
                
                # if the article has its own position in the sentence, move it here
                # otherwise it is kept as part of the NP that the identities consist of
                if ART_POS is not None:            
                    # split the identity phrase into article and the rest
                    words = identity_copy.split()
                    article = words[0]
                    identity_copy = ' '.join(words[1:])

                    # insert article at article position
                    sample[ART_POS] = article
                else:
                    article = None
                
                # insert noun at IDENTITY position
                sample[IDENTITY_POS] = identity_copy
                
                # then create one version of this per verb, if necessary
                if VERB_POS is not None:
                    for verb in VERB_LIST:
                        sample[VERB_POS] = verb
                        generated_samples.append(' '.join(sample)) # SAMPLE WITH VERB
                        samples_groups.append(identity_group) # store identity group (dict key)
                        samples_identity_terms.append(identity) # store identity term

                # then create one version of this per adjective, if necessary
                elif ADJ_POS is not None:
                    for adj in read_content(f"adjectives/{polarity}.txt"):
                        conjugations = adj.split(",") # conjugations in the order: standard, neuter, plural
                
                        # if neuter article or noun, use neuter conjugation
                        if article == "et" or sample[0].lower() == "det": 
                            adj = conjugations[1].strip()
                
                        # if plural noun phrase, use plural adjective
                        elif IS_PLURAL:
                            adj = conjugations[2].strip()
                        
                        # else use standard version            
                        else:
                            adj = conjugations[0].strip()
                            
                        # insert adjective at ADJ position and add sample to list
                        sample[ADJ_POS] = adj 
                        generated_samples.append(' '.join(sample)) # SAMPLE WITH ADJECTIVE
                        samples_groups.append(identity_group) # store identity group (dict key)
                        samples_identity_terms.append(identity) # store identity term
                        
                else: # i.e. if no verb or adjective is needed, create sample
                    generated_samples.append(' '.join(sample)) # SAMPLE WITHOUT VERB OR ADJECTIVE
                    samples_groups.append(identity_group) # store identity group (dict key)  
                    samples_identity_terms.append(identity) # store identity term   

    print(len(generated_samples), "samples created")
    
    return generated_samples, samples_groups, samples_identity_terms

In [5]:
print(len(read_content("verbs/neg_imp.txt"))*2, "verbs")
print(len(read_content("identities/female_sg.txt"))*3, "identities")
print(len(read_content("adjectives/pos.txt"))*2, "adjectives")

26 verbs
45 identities
50 adjectives


### Create samples for templates that are ALWAYS toxic such as "I hate all [identity]"

In [6]:
# open templates that are always toxic
templates = read_content("templates/always_neg.txt")
print(len(templates), "templates:")
print(templates)
print(f"\nShould get {(3*45):,} samples (3 templates * 45 identities)")

3 templates:
['IDENTITY_PL er klamme og generelt bare nederen', 'Jeg er IDENTITY_SG, og jeg hader dig og synes, du er træls', 'Jeg hader alle IDENTITY_PL']

Should get 135 samples (3 templates * 45 identities)


In [7]:
# generate samples using these templates

generated_samples, samples_groups, samples_identity_terms = generate_samples(templates)
df1 = pd.DataFrame(list(zip(generated_samples, samples_groups, samples_identity_terms)), columns=['text', 'group', 'identity_term'])
df1['toxic'] = 1 # these sentences are always non-toxic/neutral
df1.head()

135 samples created


,text,group,identity_term,toxic
0,kvinder er klamme og generelt bare nederen,F,kvinder,1
1,piger er klamme og generelt bare nederen,F,piger,1
2,damer er klamme og generelt bare nederen,F,damer,1
3,koner er klamme og generelt bare nederen,F,koner,1
4,kvindfolk er klamme og generelt bare nederen,F,kvindfolk,1


### Create samples for templates that are ALWAYS non-toxic such as "I am a/an [identity]"

In [8]:
# open templates that are always non-toxic/neutral
templates = read_content("templates/always_neu.txt")
print(len(templates), "templates:")
print(templates)
print(f"\nShould get {(3*45):,} samples (3 templates * 45 identities)")

3 templates:
['Jeg er IDENTITY_SG, spørg mig om alt', 'IDENTITY_PL er ligesom alle andre', 'Jeg er IDENTITY_SG']

Should get 135 samples (3 templates * 45 identities)


In [9]:
# generate samples using these templates

generated_samples, samples_groups, samples_identity_terms = generate_samples(templates)
df2 = pd.DataFrame(list(zip(generated_samples, samples_groups, samples_identity_terms)), columns=['text', 'group', 'identity_term'])
df2['toxic'] = 0 # these sentences are always non-toxic/neutral
df2.head()

135 samples created


,text,group,identity_term,toxic
0,Jeg er en kvinde spørg mig om alt,F,en kvinde,0
1,Jeg er en pige spørg mig om alt,F,en pige,0
2,Jeg er en dame spørg mig om alt,F,en dame,0
3,Jeg er en kone spørg mig om alt,F,en kone,0
4,Jeg er et kvindfolk spørg mig om alt,F,et kvindfolk,0


### Create samples for templates whose toxicity depend on the polarity of the verb such as "I [verb] [identities]"

In [10]:
# open templates that depend on the verb
templates = read_content("templates/verb_dependent.txt")
print(len(templates), "templates:")
print(templates)
print(f"\nShould get {(3*45*26):,} samples (3 templates * 45 identities * 26 verbs)")
print("Half should be negative/toxic, half positive/non-toxic")

3 templates:
['VERB_IMP IDENTITY_PL', 'Jeg VERB_PRS IDENTITY_PL', 'IDENTITY_PL bør VERB_PASS']

Should get 3,510 samples (3 templates * 45 identities * 26 verbs)
Half should be negative/toxic, half positive/non-toxic


In [11]:
# generate samples using these templates

# toxic/negative samples
generated_samples, samples_groups, samples_identity_terms = generate_samples(templates, "neg")
df3 = pd.DataFrame(list(zip(generated_samples, samples_groups, samples_identity_terms)), columns=['text', 'group', 'identity_term'])
df3['toxic'] = 1 
df3.head()

1755 samples created


,text,group,identity_term,toxic
0,afsky kvinder,F,kvinder,1
1,dræb kvinder,F,kvinder,1
2,eliminer kvinder,F,kvinder,1
3,had kvinder,F,kvinder,1
4,henret kvinder,F,kvinder,1


In [12]:
# non-toxic/positive samples
generated_samples, samples_groups, samples_identity_terms = generate_samples(templates, "pos")
df4 = pd.DataFrame(list(zip(generated_samples, samples_groups, samples_identity_terms)), columns=['text', 'group', 'identity_term'])
df4['toxic'] = 0 
df4.head()

1755 samples created


,text,group,identity_term,toxic
0,beundr kvinder,F,kvinder,0
1,eftertragt kvinder,F,kvinder,0
2,elsk kvinder,F,kvinder,0
3,fejr kvinder,F,kvinder,0
4,hædr kvinder,F,kvinder,0


### Create samples for templates whose toxicity depend on the polarity of the adjective such as "[identities] are [adjective]"

In [13]:
# open templates that depend on the adjective
templates = read_content("templates/adjective_dependent.txt")
print(len(templates), "templates:")
print(templates)
print(f"\nShould get {(4*45*50):,} samples (4 templates * 45 identities * 50 adjectives)")
print("Half should be negative/toxic, half positive/non-toxic")

4 templates:
['ADJECTIVE IDENTITY_PL', 'Du er ARTICLE ADJECTIVE IDENTITY_SG', 'Det er ADJECTIVE at være IDENTITY_SG', 'IDENTITY_PL er ADJECTIVE']

Should get 9,000 samples (4 templates * 45 identities * 50 adjectives)
Half should be negative/toxic, half positive/non-toxic


In [14]:
# generate samples using these templates

# toxic/negative samples
generated_samples, samples_groups, samples_identity_terms = generate_samples(templates, "neg")
df5 = pd.DataFrame(list(zip(generated_samples, samples_groups, samples_identity_terms)), columns=['text', 'group', 'identity_term'])
df5['toxic'] = 1 
df5.head()

4500 samples created


,text,group,identity_term,toxic
0,dårlige kvinder,F,kvinder,1
1,gale kvinder,F,kvinder,1
2,kriminelle kvinder,F,kvinder,1
3,onde kvinder,F,kvinder,1
4,lede kvinder,F,kvinder,1


In [15]:
# non-toxic/positive samples
generated_samples, samples_groups, samples_identity_terms = generate_samples(templates, "pos")
df6 = pd.DataFrame(list(zip(generated_samples, samples_groups, samples_identity_terms)), columns=['text', 'group', 'identity_term'])
df6['toxic'] = 0 
df6.head()

4500 samples created


,text,group,identity_term,toxic
0,sjove kvinder,F,kvinder,0
1,glade kvinder,F,kvinder,0
2,super kvinder,F,kvinder,0
3,fantastiske kvinder,F,kvinder,0
4,spændende kvinder,F,kvinder,0


### Combine all temporary dataframes into a single one and save as xlsx-file

In [16]:
# Overview of dataset 
print("Number of samples in each df:\n")
print(f"Always toxic (no verb/adj)    :    {len(df1)}")
print(f"Always non-toxic (no verb/adj):    {len(df2)}")
print(f"Toxic because of verb         :  {len(df3):,}")
print(f"Non-toxic because of verb     :  {len(df4):,}")
print(f"Toxic because of adjective    :  {len(df5):,}")
print(f"Non-toxic because of adjective:  {len(df6):,}")
print()
print(f"Total                         = {sum([len(df1), len(df2), len(df3), len(df4), len(df5), len(df6)]):,} samples")

Number of samples in each df:

Always toxic (no verb/adj)    :    135
Always non-toxic (no verb/adj):    135
Toxic because of verb         :  1,755
Non-toxic because of verb     :  1,755
Toxic because of adjective    :  4,500
Non-toxic because of adjective:  4,500

Total                         = 12,780 samples


In [21]:
# concatenate into one df
synthetic_data = pd.concat([df1, df2, df3, df4, df5, df6])
synthetic_data.drop_duplicates() # ensure that theres no duplicates
synthetic_data

,text,group,identity_term,toxic
0,kvinder er klamme og generelt bare nederen,F,kvinder,1
1,piger er klamme og generelt bare nederen,F,piger,1
2,damer er klamme og generelt bare nederen,F,damer,1
3,koner er klamme og generelt bare nederen,F,koner,1
4,kvindfolk er klamme og generelt bare nederen,F,kvindfolk,1
...,...,...,...,...
4495,genderqueer personer er suveræne,Q,genderqueer personer,0
4496,genderqueer personer er herlige,Q,genderqueer personer,0
4497,genderqueer personer er fornemme,Q,genderqueer personer,0
4498,genderqueer personer er ekstraordinære,Q,genderqueer personer,0


In [22]:
# create column with lemmas, i.e. plural == singular
def standardize_words(text:str) -> str:
    """Remove the article and 'person'. Change tempus of words that cause the lemmatizer issues.

    Args:
        text (str): text before standardization.

    Returns:
        str: text after standardization.
    """
    text = text.removeprefix("en")
    text = text.removeprefix("et")
    text = text.removesuffix("person")
    text = text.removesuffix("personer")
    text = text.strip()
    if text == "fyre":
        return "fyr"
    elif text == "mand":
        return "mænd"
    elif text == "mor":
        return "mødre"
    elif text == "tøs":
        return "tøser"
    return text

lemmatizer = lemmy.load("da")
synthetic_data['identity_lemma'] = synthetic_data['identity_term'].apply(lambda text: standardize_words(text))
synthetic_data['identity_lemma'] = synthetic_data['identity_lemma'].apply(lambda text: lemmatizer.lemmatize("", text.strip())[0]) 

print("Number of unique words before lemmatization:", synthetic_data['identity_term'].nunique())
print("Number of unique words after lemmatization:", synthetic_data['identity_lemma'].nunique())

Number of unique words before lemmatization: 90
Number of unique words after lemmatization: 45


In [23]:
# save data set
data_path = os.getcwd().removesuffix("\\create_synthetic_dataset")+"\\toxicity_detection\\data\\synthetic_data.xlsx"
synthetic_data.to_excel(data_path, index=False)

In [24]:
# how many samples are toxic and not?
print(synthetic_data['toxic'].value_counts())

toxic
1    6390
0    6390
Name: count, dtype: int64


In [25]:
# how many samples are in each identity group?
print(synthetic_data['group'].value_counts())

group
F    4260
M    4260
Q    4260
Name: count, dtype: int64


In [26]:
# create small version of dataset 
synthetic_data_small = pd.concat([df1, df2, df3, df4])
synthetic_data_small.drop_duplicates()
lemmatizer = lemmy.load("da")
synthetic_data_small['identity_lemma'] = synthetic_data_small['identity_term'].apply(lambda text: standardize_words(text))
synthetic_data_small['identity_lemma'] = synthetic_data_small['identity_lemma'].apply(lambda text: lemmatizer.lemmatize("", text.strip())[0]) 
print("Number of unique words before lemmatization:", synthetic_data_small['identity_term'].nunique())
print("Number of unique words after lemmatization:", synthetic_data_small['identity_lemma'].nunique())
data_path = os.getcwd().removesuffix("\\create_synthetic_dataset")+"\\toxicity_detection\\data\\synthetic_data_small.xlsx"
synthetic_data_small.to_excel(data_path, index=False)

Number of unique words before lemmatization: 90
Number of unique words after lemmatization: 45
